In [2]:
from questdb.ingress import Sender, TimestampNanos
import pandas as pd
from sqlalchemy import create_engine
import datetime

class QuestDBClient:
    def __init__(self, host='localhost', port=9000, username='admin', password='quest'):
        # ILP connection string for writes
        self.write_conf = f"http::addr={host}:{port};username={username};password={password};"
        # PostgreSQL connection for reads
        self.read_url = f"postgresql://{username}:{password}@{host}:8812/qdb"
        
    def insert_row(self, table_name: str, symbols: dict, columns: dict, timestamp=None):
        """Insert a single row into QuestDB"""
        try:
            with Sender.from_conf(self.write_conf) as sender:
                sender.row(
                    table_name,
                    symbols=symbols,
                    columns=columns,
                    at=timestamp or TimestampNanos.now()
                )
                sender.flush()
            return True
        except Exception as e:
            print(f"Insert error: {e}")
            return False

    def insert_dataframe(self, df: pd.DataFrame, table_name: str, timestamp_column=None):
        """Insert a pandas DataFrame into QuestDB by converting it to individual rows"""
        try:
            with Sender.from_conf(self.write_conf) as sender:
                # Convert DataFrame to rows and insert one by one
                for _, row in df.iterrows():
                    # Split columns into symbols and numeric columns
                    symbols = {col: row[col] for col in df.select_dtypes(include=['object']).columns}
                    columns = {col: row[col] for col in df.select_dtypes(exclude=['object']).columns}
                    
                    sender.row(
                        table_name,
                        symbols=symbols,
                        columns=columns,
                        at=timestamp_column or TimestampNanos.now()
                    )
                sender.flush()
            return True
        except Exception as e:
            print(f"DataFrame insert error: {e}")
            return False

    def query(self, query: str) -> pd.DataFrame:
        """Query data from QuestDB"""
        try:
            engine = create_engine(self.read_url)
            if not query.strip().endswith(';'):
                query += ';'
            with engine.connect() as conn:
                return pd.read_sql(query, conn)
        except Exception as e:
            print(f"Query error: {e}")
            return pd.DataFrame()

# Example usage:
db = QuestDBClient()

# Insert single row
symbols = {'symbol': 'BTC-USD', 'side': 'buy'}
columns = {'price': 50000.0, 'amount': 1.5}
db.insert_row('trades', symbols, columns)

# Insert DataFrame
df = pd.DataFrame({
    'symbol': ['ETH-USD', 'BTC-USD'],
    'side': ['sell', 'sell'],
    'price': [2615.54, 39269.98],
    'amount': [0.00044, 0.001]
})
db.insert_dataframe(df, 'trades')

# Query data
results = db.query("SELECT * FROM trades ORDER BY timestamp DESC LIMIT 10")
print(results)


    symbol  side     price   amount                  timestamp
0  BTC-USD  sell  39269.98  0.00100 2024-12-02 17:14:40.005123
1  ETH-USD  sell   2615.54  0.00044 2024-12-02 17:14:40.004958
2  BTC-USD   buy  50000.00  1.50000 2024-12-02 17:14:39.998405
3  BTC-USD   buy  50000.00  1.50000 2024-12-02 17:14:07.647514
